In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import numpy as np
import PIL
from functools import reduce

In [ ]:
IMG_HEIGHT = 128
IMG_WIDTH = 128
INPUT_CHANNELS = 1
import cv2
def path2img(path):
    img = PIL.Image.open(path)
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    return img
def path2imgarray(path, binary=True):

    img = PIL.Image.open(path)
    img = img.resize((IMG_WIDTH, IMG_HEIGHT))
    img = np.array(img)
    
    def binaryzation(img):
        cv_img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        return cv_img
    img = binaryzation(img) if binary else img
    img = img / 255.0
    img = img.reshape(IMG_HEIGHT, IMG_WIDTH, (1 if binary else 3) )
    return img

# Model

In [ ]:
class AutoEncoder(keras.Model):
    def __init__(self):
        layers = keras.layers
        super().__init__()
        
            
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, INPUT_CHANNELS)),
            layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),# 
            layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2), # 
        ])

        self.decoder = tf.keras.Sequential([
            layers.Conv2DTranspose(8, kernel_size=3, strides=2, activation='relu', padding='same'), # 
            layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'), # 
            layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same') # 
        ])
        
            
    def call(self, x):
        return self.decoder(self.encoder(x))

    def encode(self, x):
        return self.encoder(x)
    def decode(self, x):
        return self.decoder(x)

In [ ]:
# Create an instance of the model
autoencoder = AutoEncoder()

trainLoss = tf.keras.metrics.Mean(name='trainLoss')
valLoss = tf.keras.metrics.Mean(name='valLoss')
calLoss = tf.keras.losses.MeanSquaredError()
l1 = tf.keras.regularizers.L1(l1=0.01)

autoencoder.build(input_shape=(None, IMG_HEIGHT, IMG_WIDTH, INPUT_CHANNELS))
autoencoder.summary()
# 查看每一层
for layer in autoencoder.encoder.layers:
    print(layer.name, layer.output_shape)
for layer in autoencoder.decoder.layers:
    print(layer.name, layer.output_shape)

In [ ]:
def getTrainStap(model):
    optimizer = tf.keras.optimizers.Adam()
    @tf.function
    def trainStep(images, labels):
        with tf.GradientTape() as tape:
            predictions = model.encode(images)
            predictions = model.decode(predictions)

            loss = calLoss(labels, predictions) # * l1
        # print(loss)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        trainLoss(loss)
    return trainStep
def getValStep(model):
    @tf.function
    def valStep(images, labels):
        predictions = model(images, training=False)
        loss = calLoss(labels, predictions)

        valLoss(loss)
    return valStep

# Data

In [ ]:
BATCH_SIZE = 50
trainNoiseDir = './dataset/train/noise/'
trainNoNoiseDir = './dataset/train/noNoise/'
testNoiseDir = './dataset/test/noise/'
testNoNoiseDir = './dataset/test/noNoise/'
# noise 和 noNoise 是输入和标签
dataset = tf.data.Dataset.from_tensor_slices(
    ([path2imgarray(path, binary=False) for path in tf.io.gfile.glob(trainNoiseDir + '*.jpg')], 
    [path2imgarray(path, binary=True) for path in tf.io.gfile.glob(trainNoNoiseDir + '*.jpg')])
)
dataset = dataset.shuffle(buffer_size=1000)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

valDataset = tf.data.Dataset.from_tensor_slices(
    ([path2imgarray(path, binary=False) for path in tf.io.gfile.glob(testNoiseDir + '*.jpg')],
    [path2imgarray(path, binary=True) for path in tf.io.gfile.glob(testNoNoiseDir + '*.jpg')])
)

valDataset = valDataset.shuffle(buffer_size=1000)
valDataset = valDataset.batch(BATCH_SIZE)
valDataset = valDataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
print(len(dataset))


# Train

In [ ]:
import time,random
# train face1
trainStep = getTrainStap(autoencoder)
valiStep = getValStep(autoencoder)
EPOCHS = 2000

lossHistory = []
epochsRange = []

valLossHistory = []

start = time.time()
for epoch in range(EPOCHS):
    trainLoss.reset_states()
    valLoss.reset_states()
        
    for images, labels in dataset:
        trainStep(images, labels)
    
    # random validate
    for images, labels in valDataset.take(10):
        valiStep(images,labels)
        
    


    
    print(
        f'\nEpoch {epoch + 1}, '
        f'\nTrain      Loss: {trainLoss.result()}, '
        f'\nValidation Loss: {valLoss.result()}'
        f' - {int((time.time() - start) / (epoch + 1) * (EPOCHS - epoch)) / 60} minutes left, '
    )
    lossHistory.append(trainLoss.result())
    valLossHistory.append(valLoss.result())
    epochsRange.append(epoch)

plt.plot(epochsRange[100:], lossHistory[100:], label='Trainning Loss')
plt.plot(epochsRange[100:], valLossHistory[100:], label='Validation Loss')
# 图例
plt.legend(loc='upper right')
plt.show()


In [ ]:
minloss = np.min(valLossHistory)
epochln = len(epochsRange)
for i in range(10):
  plt.figure(figsize=(5, 5), dpi=200)
  plt.plot([0, epochln], [minloss,minloss],linestyle='--',color='black')
  # text minloss
  plt.text(0, minloss, '%.5f'%minloss, fontsize=10)
  plt.plot(epochsRange[10 * i:], lossHistory[10 * i:], label='Training Loss')
  plt.plot(epochsRange[10 * i:], valLossHistory[10 * i:], label='Validation Loss')
  # 图例
  plt.legend(loc='upper right')
  plt.show()


In [ ]:
autoencoder.save_weights('./captchModel.h5')
#autoencoder.load_weights('./captchModel.h5')

In [ ]:
fig,ax = plt.subplots(ncols=6,nrows=2)
fig.dpi = 200
trainData = [*dataset.take(1)][0]
valData = [*valDataset.take(1)][0]
trainInputImg = trainData[0][:1]
trainTargetImg = trainData[1][:1]
valInputImg = valData[0][:1] 
valTargetImg = valData[1][:1]
ax[0][0].imshow(trainInputImg[0])
ax[0][1].imshow(trainTargetImg[0])
ax[0][2].imshow(autoencoder(trainInputImg,training=False)[0])
ax[1][0].imshow(valInputImg[0])
ax[1][1].imshow(valTargetImg[0])
ax[1][2].imshow(autoencoder(valInputImg,training=False)[0])
import cv2
# 二值化
def binaryzation(img):
    cv_img = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    cv_img = cv2.threshold(cv_img,190,255,cv2.THRESH_BINARY_INV)[1]
    return cv_img
# read img from np float
# float to int
def img2int(img):
    img *= 255
    img = img.astype(np.uint8)
    return img
ax[0][3].imshow(np.array(autoencoder(trainInputImg,training=False)[0]))
ax[1][3].imshow(np.array(autoencoder(valInputImg,training=False)[0]))
# 中值滤波
def medianBlur(img):
    img = cv2.medianBlur(img,5)
    return img
ax[0][4].imshow(medianBlur(np.array(autoencoder(trainInputImg,training=False)[0])))
ax[1][4].imshow(medianBlur(np.array(autoencoder(valInputImg,training=False)[0])))
ax[0][5].imshow(medianBlur(img2int(np.array(trainInputImg[0]))))
ax[1][5].imshow(medianBlur(img2int(np.array(valInputImg[0]))))

In [ ]:
_100Model = AutoEncoder()
_400Model = AutoEncoder()
_100Model.build(input_shape=(None,IMG_HEIGHT,IMG_WIDTH,1))
_400Model.build(input_shape=(None,IMG_HEIGHT,IMG_WIDTH,1))
_100Model.load_weights('./100dataset2000epoch.h5')
_400Model.load_weights('./400dataset2000epoch.h5')
trainSet = [*zip(
  [path2imgarray(path, binary=False) for path in tf.io.gfile.glob(testNoiseDir + '*.jpg')],
  [path2imgarray(path, binary=True) for path in tf.io.gfile.glob(testNoiseDir + '*.jpg')]
)]
random.shuffle(trainSet)
trainSet = trainSet[:5]
fig, ax =plt.subplots(ncols=5,nrows=2,figsize=(13, 4), dpi=200)
idx = 0
for origin, binary in trainSet:
  # 去除刻度
    ax[0][idx].xaxis.set_visible(False)
    ax[0][idx].yaxis.set_visible(False)
    
    ax[0][idx].imshow(origin)
    ax[1][idx].imshow(_100Model(binary.reshape(1,IMG_HEIGHT, IMG_WIDTH,1),training=False)[0])
    idx += 1
fig, ax =plt.subplots(ncols=5,nrows=2,figsize=(13, 4), dpi=200)
idx = 0
for origin, binary in trainSet:
  # 去除刻度
    ax[0][idx].xaxis.set_visible(False)
    ax[0][idx].yaxis.set_visible(False)
    ax[0][idx].imshow(origin)
    ax[1][idx].imshow(_400Model(binary.reshape(1,IMG_HEIGHT, IMG_WIDTH,1),training=False)[0])
    idx += 1


In [ ]:
autoencoder = AutoEncoder()
autoencoder.build(input_shape=(None,IMG_HEIGHT,IMG_WIDTH,1))
autoencoder.load_weights('./100dataset2000epoch.h5')
img = path2imgarray('./noise-BNEZ.jpg', binary=False)
binary = path2imgarray('./noise-BNEZ.jpg', binary=True)
img = img.reshape((1,IMG_HEIGHT,IMG_WIDTH,3))
binary = binary.reshape((1,IMG_HEIGHT,IMG_WIDTH,1)) 
fig,ax = plt.subplots(ncols=2,nrows=1)
fig.dpi = 100
ax[0].imshow(img[0])
ax[1].imshow(
    autoencoder(binary,training=False)[0]
)
# conv2d (None, 64, 64, 16)
# conv2d_1 (None, 32, 32, 8)
# conv2d_transpose (None, 64, 64, 8)
# conv2d_transpose_1 (None, 128, 128, 16)
# conv2d_2 (None, 128, 128, 1)
_6416out = autoencoder.encoder.layers[0](binary)
_328out = autoencoder.encoder.layers[1](_6416out)
_648out = autoencoder.decoder.layers[0](_328out)
_12816out = autoencoder.decoder.layers[1](_648out)
out = autoencoder.decoder.layers[2](_12816out)
fig, ax = plt.subplots(ncols=4,nrows=4,dpi =200)
# title
ax[0][0].set_title('encoder layer1 (None, 64, 64, 16)', fontsize=8,)
for i in range(16):
    ax[i//4][i%4].imshow(_6416out[0][:,:,i])
    ax[i//4][i%4].xaxis.set_visible(False)
    
plt.show()

fig, ax = plt.subplots(ncols=4,nrows=2,dpi =200)
# title
ax[0][0].set_title('encoder layer2 (None, 32, 32, 8)', fontsize=8,)
for i in range(8):
    ax[i//4][i%4].imshow(_328out[0][:,:,i])
    ax[i//4][i%4].xaxis.set_visible(False)

plt.show()
# title
fig, ax = plt.subplots(ncols=4,nrows=2,dpi =200)
ax[0][0].set_title('decoder layer1 (None, 64, 64, 8)', fontsize=8,)
for i in range(8):
    ax[i//4][i%4].imshow(_648out[0][:,:,i])
    ax[i//4][i%4].xaxis.set_visible(False)
plt.show()
fig,ax = plt.subplots(ncols=4,nrows=4,dpi =200)
# title
ax[0][0].set_title('decoder layer2 (None, 128, 128, 16)', fontsize=8,)
for i in range(16):
    ax[i//4][i%4].imshow(_12816out[0][:,:,i])
    ax[i//4][i%4].xaxis.set_visible(False)